In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.648474931716919, Final Batch Loss: 2.3196909427642822
Epoch 2, Loss: 4.623358488082886, Final Batch Loss: 2.307652473449707
Epoch 3, Loss: 4.5970988273620605, Final Batch Loss: 2.2963767051696777
Epoch 4, Loss: 4.576807498931885, Final Batch Loss: 2.282341480255127
Epoch 5, Loss: 4.567737102508545, Final Batch Loss: 2.2840986251831055
Epoch 6, Loss: 4.5502753257751465, Final Batch Loss: 2.271935224533081
Epoch 7, Loss: 4.533130884170532, Final Batch Loss: 2.2686078548431396
Epoch 8, Loss: 4.5102198123931885, Final Batch Loss: 2.2504889965057373
Epoch 9, Loss: 4.499437570571899, Final Batch Loss: 2.251394271850586
Epoch 10, Loss: 4.465226888656616, Final Batch Loss: 2.22273325920105
Epoch 11, Loss: 4.462246656417847, Final Batch Loss: 2.2244443893432617
Epoch 12, Loss: 4.436897039413452, Final Batch Loss: 2.210871696472168
Epoch 13, Loss: 4.4104626178741455, Final Batch Loss: 2.192931652069092
Epoch 14, Loss: 4.388186693191528, Final Batch Loss: 2.1845695972442627
Epoch

Epoch 126, Loss: 0.4864729344844818, Final Batch Loss: 0.24793638288974762
Epoch 127, Loss: 0.4893082231283188, Final Batch Loss: 0.2616182565689087
Epoch 128, Loss: 0.4288881868124008, Final Batch Loss: 0.2158467173576355
Epoch 129, Loss: 0.46614232659339905, Final Batch Loss: 0.23878240585327148
Epoch 130, Loss: 0.5134861171245575, Final Batch Loss: 0.2819426357746124
Epoch 131, Loss: 0.4315120428800583, Final Batch Loss: 0.22077064216136932
Epoch 132, Loss: 0.33971795439720154, Final Batch Loss: 0.17709988355636597
Epoch 133, Loss: 0.41290001571178436, Final Batch Loss: 0.20059797167778015
Epoch 134, Loss: 0.36272308230400085, Final Batch Loss: 0.1631377786397934
Epoch 135, Loss: 0.33174146711826324, Final Batch Loss: 0.1414182335138321
Epoch 136, Loss: 0.35808880627155304, Final Batch Loss: 0.1647946983575821
Epoch 137, Loss: 0.44676582515239716, Final Batch Loss: 0.23249658942222595
Epoch 138, Loss: 0.38004089891910553, Final Batch Loss: 0.22184085845947266
Epoch 139, Loss: 0.3702

Epoch 247, Loss: 0.12004340812563896, Final Batch Loss: 0.07239237427711487
Epoch 248, Loss: 0.16316915675997734, Final Batch Loss: 0.10284240543842316
Epoch 249, Loss: 0.22576454281806946, Final Batch Loss: 0.1565331667661667
Epoch 250, Loss: 0.16041389852762222, Final Batch Loss: 0.09881991147994995
Epoch 251, Loss: 0.17883402481675148, Final Batch Loss: 0.1201702132821083
Epoch 252, Loss: 0.15341492742300034, Final Batch Loss: 0.06273236870765686
Epoch 253, Loss: 0.19720254093408585, Final Batch Loss: 0.06556610018014908
Epoch 254, Loss: 0.16605251654982567, Final Batch Loss: 0.10614322125911713
Epoch 255, Loss: 0.17652291059494019, Final Batch Loss: 0.06298456341028214
Epoch 256, Loss: 0.22017325460910797, Final Batch Loss: 0.14905241131782532
Epoch 257, Loss: 0.1613793820142746, Final Batch Loss: 0.09042596817016602
Epoch 258, Loss: 0.1259823963046074, Final Batch Loss: 0.05318401753902435
Epoch 259, Loss: 0.13370738178491592, Final Batch Loss: 0.043306365609169006
Epoch 260, Loss

Epoch 368, Loss: 0.10020487383008003, Final Batch Loss: 0.053345367312431335
Epoch 369, Loss: 0.09479885175824165, Final Batch Loss: 0.06359683722257614
Epoch 370, Loss: 0.08013887703418732, Final Batch Loss: 0.03587422892451286
Epoch 371, Loss: 0.10785745084285736, Final Batch Loss: 0.04804714024066925
Epoch 372, Loss: 0.09509821608662605, Final Batch Loss: 0.023323874920606613
Epoch 373, Loss: 0.06272061727941036, Final Batch Loss: 0.046783894300460815
Epoch 374, Loss: 0.07934439927339554, Final Batch Loss: 0.03987250477075577
Epoch 375, Loss: 0.0875556692481041, Final Batch Loss: 0.029867321252822876
Epoch 376, Loss: 0.13045365177094936, Final Batch Loss: 0.10805971175432205
Epoch 377, Loss: 0.06369110569357872, Final Batch Loss: 0.015738286077976227
Epoch 378, Loss: 0.05603618547320366, Final Batch Loss: 0.02274627611041069
Epoch 379, Loss: 0.08155519515275955, Final Batch Loss: 0.05313856899738312
Epoch 380, Loss: 0.09957064315676689, Final Batch Loss: 0.032558586448431015
Epoch 3

Epoch 492, Loss: 0.08257126808166504, Final Batch Loss: 0.041352737694978714
Epoch 493, Loss: 0.07686110958456993, Final Batch Loss: 0.031948529183864594
Epoch 494, Loss: 0.08410773053765297, Final Batch Loss: 0.0563068762421608
Epoch 495, Loss: 0.055105216801166534, Final Batch Loss: 0.0249064601957798
Epoch 496, Loss: 0.05474236235022545, Final Batch Loss: 0.016632702201604843
Epoch 497, Loss: 0.04013371840119362, Final Batch Loss: 0.014959702268242836
Epoch 498, Loss: 0.06276905164122581, Final Batch Loss: 0.03872980177402496
Epoch 499, Loss: 0.09667671658098698, Final Batch Loss: 0.07094991952180862
Epoch 500, Loss: 0.06409794837236404, Final Batch Loss: 0.020695481449365616
Epoch 501, Loss: 0.06751013919711113, Final Batch Loss: 0.031506430357694626
Epoch 502, Loss: 0.09196069836616516, Final Batch Loss: 0.04866904392838478
Epoch 503, Loss: 0.08641378208994865, Final Batch Loss: 0.05728285759687424
Epoch 504, Loss: 0.07782241702079773, Final Batch Loss: 0.04381241276860237
Epoch 5

...

Epoch 4564, Loss: 0.017969442564208293, Final Batch Loss: 0.017933759838342667
Epoch 4565, Loss: 0.00032796907908050343, Final Batch Loss: 8.961988351074979e-05
Epoch 4566, Loss: 0.0012461579899536446, Final Batch Loss: 0.0011089702602475882
Epoch 4567, Loss: 6.650272462138673e-05, Final Batch Loss: 1.3615229363495018e-05
Epoch 4568, Loss: 7.337343703284205e-05, Final Batch Loss: 7.793088911967061e-07
Epoch 4569, Loss: 1.902187477753614e-05, Final Batch Loss: 1.5164044270932209e-05
Epoch 4570, Loss: 0.0003159206680720672, Final Batch Loss: 0.00014418212231248617
Epoch 4571, Loss: 0.00021645530250680167, Final Batch Loss: 0.00018673385784495622
Epoch 4572, Loss: 0.0010163805272895843, Final Batch Loss: 0.0003935839340556413
Epoch 4573, Loss: 0.0009560619073454291, Final Batch Loss: 0.0006218365742824972
Epoch 4574, Loss: 0.0012710140499621048, Final Batch Loss: 0.0012598497560247779
Epoch 4575, Loss: 0.0004914038581773639, Final Batch Loss: 6.697711069136858e-05
Epoch 4576, Loss: 0.0003

Epoch 4664, Loss: 0.0009643413941375911, Final Batch Loss: 0.0006901034503243864
Epoch 4665, Loss: 0.0004332938260631636, Final Batch Loss: 0.00027274645981378853
Epoch 4666, Loss: 0.00042429202585481107, Final Batch Loss: 0.00019615325436461717
Epoch 4667, Loss: 0.0067068220232613385, Final Batch Loss: 0.006550215650349855
Epoch 4668, Loss: 0.04443028042442165, Final Batch Loss: 6.977314478717744e-05
Epoch 4669, Loss: 9.986877921619453e-05, Final Batch Loss: 4.4490861910162494e-05
Epoch 4670, Loss: 0.0002871125179808587, Final Batch Loss: 6.639385537710041e-05
Epoch 4671, Loss: 9.186784518533386e-05, Final Batch Loss: 3.7615060136886314e-05
Epoch 4672, Loss: 0.00025071892014238983, Final Batch Loss: 7.206600275821984e-05
Epoch 4673, Loss: 0.007941298885270953, Final Batch Loss: 0.0073255691677331924
Epoch 4674, Loss: 0.00011128651021863334, Final Batch Loss: 8.156487456290051e-05
Epoch 4675, Loss: 0.0005612572349491529, Final Batch Loss: 0.0004682815633714199
Epoch 4676, Loss: 0.00090

Epoch 4769, Loss: 0.00019551037985365838, Final Batch Loss: 3.341154661029577e-05
Epoch 4770, Loss: 0.0003263541220803745, Final Batch Loss: 0.00028134274180047214
Epoch 4771, Loss: 0.0001102719015761977, Final Batch Loss: 7.764681868138723e-06
Epoch 4772, Loss: 0.0014539374969899654, Final Batch Loss: 0.0009649060666561127
Epoch 4773, Loss: 0.0002751837237155996, Final Batch Loss: 0.00022520158381666988
Epoch 4774, Loss: 0.00016963817506621126, Final Batch Loss: 4.113573595532216e-06
Epoch 4775, Loss: 0.0021364345147958375, Final Batch Loss: 7.355439265666064e-06
Epoch 4776, Loss: 5.387611599871889e-05, Final Batch Loss: 7.226586603792384e-06
Epoch 4777, Loss: 0.003949650632421253, Final Batch Loss: 0.003938314970582724
Epoch 4778, Loss: 9.808674622036051e-05, Final Batch Loss: 1.4716320947627537e-05
Epoch 4779, Loss: 0.0005211043171584606, Final Batch Loss: 9.131856495514512e-05
Epoch 4780, Loss: 0.000754823035094887, Final Batch Loss: 0.0005963873118162155
Epoch 4781, Loss: 0.000206

Epoch 4871, Loss: 2.94776955342968e-05, Final Batch Loss: 1.4494933566311374e-05
Epoch 4872, Loss: 8.618009178462671e-05, Final Batch Loss: 8.201852324418724e-05
Epoch 4873, Loss: 0.0009867958269751398, Final Batch Loss: 1.5794195860507898e-05
Epoch 4874, Loss: 0.00012963916105945827, Final Batch Loss: 0.00011984672164544463
Epoch 4875, Loss: 0.0004371376853669062, Final Batch Loss: 8.627046190667897e-05
Epoch 4876, Loss: 0.00026903641264652833, Final Batch Loss: 9.682396921562031e-05
Epoch 4877, Loss: 0.008623289177194238, Final Batch Loss: 0.0012270587030798197
Epoch 4878, Loss: 0.00028374743124004453, Final Batch Loss: 0.0002081408747471869
Epoch 4879, Loss: 0.0001678028238529805, Final Batch Loss: 0.00011019869998563081
Epoch 4880, Loss: 0.0014514115027850494, Final Batch Loss: 0.0013626719592139125
Epoch 4881, Loss: 0.0011212934500690608, Final Batch Loss: 6.459000815084437e-06
Epoch 4882, Loss: 0.0002545963870943524, Final Batch Loss: 0.00020905700512230396
Epoch 4883, Loss: 0.00

Epoch 4972, Loss: 0.0011068337480537593, Final Batch Loss: 0.00015148607781156898
Epoch 4973, Loss: 5.244573185336776e-05, Final Batch Loss: 2.5341883883811533e-05
Epoch 4974, Loss: 0.0006102504266891629, Final Batch Loss: 7.088927668519318e-05
Epoch 4975, Loss: 0.053342120721936226, Final Batch Loss: 0.0438234768807888
Epoch 4976, Loss: 0.00010477547766640782, Final Batch Loss: 2.320507337572053e-05
Epoch 4977, Loss: 0.0024040243442868814, Final Batch Loss: 0.0022539307828992605
Epoch 4978, Loss: 0.0004416939045768231, Final Batch Loss: 0.00014741247287020087
Epoch 4979, Loss: 0.000994931921013631, Final Batch Loss: 9.390486229676753e-05
Epoch 4980, Loss: 0.004964681051205844, Final Batch Loss: 0.0007880820776335895
Epoch 4981, Loss: 0.011447693570517004, Final Batch Loss: 0.0004784570774063468
Epoch 4982, Loss: 0.003932070641894825, Final Batch Loss: 0.00016821474127937108
Epoch 4983, Loss: 0.0016932617290876806, Final Batch Loss: 0.0013645726721733809
Epoch 4984, Loss: 0.00027483156

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[43  0  0]
 [ 1 29  0]
 [ 0  0 27]]
              precision    recall  f1-score   support

           0    0.97727   1.00000   0.98851        43
           1    1.00000   0.96667   0.98305        30
           2    1.00000   1.00000   1.00000        27

    accuracy                        0.99000       100
   macro avg    0.99242   0.98889   0.99052       100
weighted avg    0.99023   0.99000   0.98997       100



In [20]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 3)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 3)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 3) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [21]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[30  0  0]
 [ 0 30  0]
 [ 0  0 30]]
              precision    recall  f1-score   support

         0.0    1.00000   1.00000   1.00000        30
         1.0    1.00000   1.00000   1.00000        30
         2.0    1.00000   1.00000   1.00000        30

    accuracy                        1.00000        90
   macro avg    1.00000   1.00000   1.00000        90
weighted avg    1.00000   1.00000   1.00000        90



# User Classifier

In [22]:
activities = [1, 3, 4]
users = [1, 3, 5]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [23]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [25]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.609975576400757, Final Batch Loss: 2.3012683391571045
Epoch 2, Loss: 4.586054086685181, Final Batch Loss: 2.289593458175659
Epoch 3, Loss: 4.565965414047241, Final Batch Loss: 2.2710883617401123
Epoch 4, Loss: 4.54191780090332, Final Batch Loss: 2.2621238231658936
Epoch 5, Loss: 4.524929046630859, Final Batch Loss: 2.256298303604126
Epoch 6, Loss: 4.510240793228149, Final Batch Loss: 2.2469875812530518
Epoch 7, Loss: 4.500760078430176, Final Batch Loss: 2.254378318786621
Epoch 8, Loss: 4.479606866836548, Final Batch Loss: 2.2345523834228516
Epoch 9, Loss: 4.470067024230957, Final Batch Loss: 2.2366249561309814
Epoch 10, Loss: 4.462719440460205, Final Batch Loss: 2.241452932357788
Epoch 11, Loss: 4.446505546569824, Final Batch Loss: 2.2324743270874023
Epoch 12, Loss: 4.432318687438965, Final Batch Loss: 2.2143406867980957
Epoch 13, Loss: 4.424461841583252, Final Batch Loss: 2.21889591217041
Epoch 14, Loss: 4.409650802612305, Final Batch Loss: 2.1961886882781982
Epoch 15

Epoch 132, Loss: 2.0259915590286255, Final Batch Loss: 1.033648133277893
Epoch 133, Loss: 1.9803404211997986, Final Batch Loss: 0.9971688389778137
Epoch 134, Loss: 1.987761914730072, Final Batch Loss: 0.9940436482429504
Epoch 135, Loss: 1.9792904257774353, Final Batch Loss: 1.003831148147583
Epoch 136, Loss: 2.013852894306183, Final Batch Loss: 1.018222689628601
Epoch 137, Loss: 2.002758264541626, Final Batch Loss: 1.0083550214767456
Epoch 138, Loss: 1.9693964123725891, Final Batch Loss: 1.0003585815429688
Epoch 139, Loss: 1.9465965628623962, Final Batch Loss: 0.9644733667373657
Epoch 140, Loss: 1.9141857624053955, Final Batch Loss: 0.9590333700180054
Epoch 141, Loss: 1.9132484197616577, Final Batch Loss: 0.9690902829170227
Epoch 142, Loss: 1.9015824794769287, Final Batch Loss: 0.9580671787261963
Epoch 143, Loss: 1.9448322057724, Final Batch Loss: 0.9889278411865234
Epoch 144, Loss: 1.8992788195610046, Final Batch Loss: 0.9320835471153259
Epoch 145, Loss: 1.8529087901115417, Final Batc

Epoch 248, Loss: 1.4582616090774536, Final Batch Loss: 0.7547324895858765
Epoch 249, Loss: 1.395854890346527, Final Batch Loss: 0.7345539331436157
Epoch 250, Loss: 1.43478262424469, Final Batch Loss: 0.7398431301116943
Epoch 251, Loss: 1.4122412204742432, Final Batch Loss: 0.7184169292449951
Epoch 252, Loss: 1.4265809655189514, Final Batch Loss: 0.7301911115646362
Epoch 253, Loss: 1.2865977883338928, Final Batch Loss: 0.6051340699195862
Epoch 254, Loss: 1.460637092590332, Final Batch Loss: 0.7156751155853271
Epoch 255, Loss: 1.4376250505447388, Final Batch Loss: 0.7581589221954346
Epoch 256, Loss: 1.3889211416244507, Final Batch Loss: 0.6728999614715576
Epoch 257, Loss: 1.374009609222412, Final Batch Loss: 0.6949172616004944
Epoch 258, Loss: 1.415930986404419, Final Batch Loss: 0.7359036207199097
Epoch 259, Loss: 1.350686490535736, Final Batch Loss: 0.6223907470703125
Epoch 260, Loss: 1.3143553733825684, Final Batch Loss: 0.6219771504402161
Epoch 261, Loss: 1.3402056694030762, Final Ba

Epoch 367, Loss: 1.098425805568695, Final Batch Loss: 0.5686152577400208
Epoch 368, Loss: 1.0611799359321594, Final Batch Loss: 0.4727662205696106
Epoch 369, Loss: 1.1088201403617859, Final Batch Loss: 0.5021417140960693
Epoch 370, Loss: 1.0859997272491455, Final Batch Loss: 0.5559614896774292
Epoch 371, Loss: 1.1062759757041931, Final Batch Loss: 0.5458946824073792
Epoch 372, Loss: 1.136043667793274, Final Batch Loss: 0.5885178446769714
Epoch 373, Loss: 1.1081672310829163, Final Batch Loss: 0.5706725120544434
Epoch 374, Loss: 1.1121897101402283, Final Batch Loss: 0.5888597965240479
Epoch 375, Loss: 1.1035274863243103, Final Batch Loss: 0.5676025748252869
Epoch 376, Loss: 1.1042283773422241, Final Batch Loss: 0.5526482462882996
Epoch 377, Loss: 1.1363344192504883, Final Batch Loss: 0.5453546047210693
Epoch 378, Loss: 1.0405402779579163, Final Batch Loss: 0.4978369474411011
Epoch 379, Loss: 1.0993768572807312, Final Batch Loss: 0.584067702293396
Epoch 380, Loss: 1.0485073924064636, Fina

Epoch 487, Loss: 0.7910944223403931, Final Batch Loss: 0.4001050293445587
Epoch 488, Loss: 0.7390953600406647, Final Batch Loss: 0.314117431640625
Epoch 489, Loss: 0.8749294877052307, Final Batch Loss: 0.40174782276153564
Epoch 490, Loss: 0.7641454935073853, Final Batch Loss: 0.3439861834049225
Epoch 491, Loss: 0.8567964434623718, Final Batch Loss: 0.49128079414367676
Epoch 492, Loss: 0.7960097789764404, Final Batch Loss: 0.3891165256500244
Epoch 493, Loss: 0.8238874077796936, Final Batch Loss: 0.41962313652038574
Epoch 494, Loss: 0.8223270177841187, Final Batch Loss: 0.4308856725692749
Epoch 495, Loss: 0.8633500933647156, Final Batch Loss: 0.44941842555999756
Epoch 496, Loss: 0.7281408905982971, Final Batch Loss: 0.43461108207702637
Epoch 497, Loss: 0.7497831284999847, Final Batch Loss: 0.4015522599220276
Epoch 498, Loss: 0.7210688591003418, Final Batch Loss: 0.36084797978401184
Epoch 499, Loss: 0.7334306836128235, Final Batch Loss: 0.34532445669174194
Epoch 500, Loss: 0.7669994235038

...

Epoch 4501, Loss: 0.07493017613887787, Final Batch Loss: 0.05326753482222557
Epoch 4502, Loss: 0.03214479051530361, Final Batch Loss: 0.012252258136868477
Epoch 4503, Loss: 0.058600117452442646, Final Batch Loss: 0.04380488395690918
Epoch 4504, Loss: 0.06324422918260098, Final Batch Loss: 0.0476059727370739
Epoch 4505, Loss: 0.06626600213348866, Final Batch Loss: 0.04180362820625305
Epoch 4506, Loss: 0.06503564305603504, Final Batch Loss: 0.04105044901371002
Epoch 4507, Loss: 0.0783156044781208, Final Batch Loss: 0.04481012374162674
Epoch 4508, Loss: 0.10140317678451538, Final Batch Loss: 0.056269969791173935
Epoch 4509, Loss: 0.09252814762294292, Final Batch Loss: 0.028562812134623528
Epoch 4510, Loss: 0.07303647696971893, Final Batch Loss: 0.033692874014377594
Epoch 4511, Loss: 0.03445618599653244, Final Batch Loss: 0.02317764423787594
Epoch 4512, Loss: 0.1059323139488697, Final Batch Loss: 0.0596490278840065
Epoch 4513, Loss: 0.10664783045649529, Final Batch Loss: 0.0858652442693710

Epoch 4616, Loss: 0.07130728289484978, Final Batch Loss: 0.008762475103139877
Epoch 4617, Loss: 0.0512765608727932, Final Batch Loss: 0.02867867425084114
Epoch 4618, Loss: 0.02848115097731352, Final Batch Loss: 0.005206049419939518
Epoch 4619, Loss: 0.11132221668958664, Final Batch Loss: 0.09933539479970932
Epoch 4620, Loss: 0.026818384416401386, Final Batch Loss: 0.01654530130326748
Epoch 4621, Loss: 0.030997445806860924, Final Batch Loss: 0.01257079653441906
Epoch 4622, Loss: 0.053800662979483604, Final Batch Loss: 0.009787211194634438
Epoch 4623, Loss: 0.05948742851614952, Final Batch Loss: 0.0344746895134449
Epoch 4624, Loss: 0.051534658297896385, Final Batch Loss: 0.01941411755979061
Epoch 4625, Loss: 0.08016730844974518, Final Batch Loss: 0.025857068598270416
Epoch 4626, Loss: 0.07558638975024223, Final Batch Loss: 0.04543877765536308
Epoch 4627, Loss: 0.0277286097407341, Final Batch Loss: 0.019154636189341545
Epoch 4628, Loss: 0.08442270010709763, Final Batch Loss: 0.05315871536

Epoch 4736, Loss: 0.023892766796052456, Final Batch Loss: 0.009712906554341316
Epoch 4737, Loss: 0.02286219224333763, Final Batch Loss: 0.013004112057387829
Epoch 4738, Loss: 0.09170351177453995, Final Batch Loss: 0.05783659592270851
Epoch 4739, Loss: 0.02950519137084484, Final Batch Loss: 0.014592417515814304
Epoch 4740, Loss: 0.029563426971435547, Final Batch Loss: 0.010636512190103531
Epoch 4741, Loss: 0.043602682650089264, Final Batch Loss: 0.02467632293701172
Epoch 4742, Loss: 0.03249047510325909, Final Batch Loss: 0.017886560410261154
Epoch 4743, Loss: 0.02047168742865324, Final Batch Loss: 0.008971049450337887
Epoch 4744, Loss: 0.02804542426019907, Final Batch Loss: 0.014749638736248016
Epoch 4745, Loss: 0.016649825032800436, Final Batch Loss: 0.009132507257163525
Epoch 4746, Loss: 0.03269634488970041, Final Batch Loss: 0.015233934856951237
Epoch 4747, Loss: 0.05521832965314388, Final Batch Loss: 0.03759745508432388
Epoch 4748, Loss: 0.09433901682496071, Final Batch Loss: 0.0547

Epoch 4847, Loss: 0.1210121437907219, Final Batch Loss: 0.08924856781959534
Epoch 4848, Loss: 0.04457064997404814, Final Batch Loss: 0.014864779077470303
Epoch 4849, Loss: 0.040714507922530174, Final Batch Loss: 0.0077688079327344894
Epoch 4850, Loss: 0.03817863576114178, Final Batch Loss: 0.018538590520620346
Epoch 4851, Loss: 0.08603058010339737, Final Batch Loss: 0.050624679774045944
Epoch 4852, Loss: 0.034944978542625904, Final Batch Loss: 0.020870478823781013
Epoch 4853, Loss: 0.061305390670895576, Final Batch Loss: 0.021221959963440895
Epoch 4854, Loss: 0.10292516648769379, Final Batch Loss: 0.053258635103702545
Epoch 4855, Loss: 0.035023318603634834, Final Batch Loss: 0.008201615884900093
Epoch 4856, Loss: 0.05563250556588173, Final Batch Loss: 0.031843703240156174
Epoch 4857, Loss: 0.09700311161577702, Final Batch Loss: 0.07600434869527817
Epoch 4858, Loss: 0.028825287707149982, Final Batch Loss: 0.008551637642085552
Epoch 4859, Loss: 0.09218345209956169, Final Batch Loss: 0.05

Epoch 4962, Loss: 0.036525264382362366, Final Batch Loss: 0.008763235062360764
Epoch 4963, Loss: 0.029684318462386727, Final Batch Loss: 0.02736683189868927
Epoch 4964, Loss: 0.04699908336624503, Final Batch Loss: 0.005218998994678259
Epoch 4965, Loss: 0.14282003231346607, Final Batch Loss: 0.1236017718911171
Epoch 4966, Loss: 0.06032184977084398, Final Batch Loss: 0.048999737948179245
Epoch 4967, Loss: 0.09121967852115631, Final Batch Loss: 0.045387908816337585
Epoch 4968, Loss: 0.0936163179576397, Final Batch Loss: 0.03331755846738815
Epoch 4969, Loss: 0.09759348258376122, Final Batch Loss: 0.01847795769572258
Epoch 4970, Loss: 0.059398023411631584, Final Batch Loss: 0.030512263998389244
Epoch 4971, Loss: 0.04900637827813625, Final Batch Loss: 0.015856536105275154
Epoch 4972, Loss: 0.0330607620999217, Final Batch Loss: 0.014416036196053028
Epoch 4973, Loss: 0.02891141176223755, Final Batch Loss: 0.009261097759008408
Epoch 4974, Loss: 0.04580942168831825, Final Batch Loss: 0.019608864

In [26]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[35  0  0]
 [ 0 29  0]
 [ 0  1 35]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        35
           1    0.96667   1.00000   0.98305        29
           2    1.00000   0.97222   0.98592        36

    accuracy                        0.99000       100
   macro avg    0.98889   0.99074   0.98966       100
weighted avg    0.99033   0.99000   0.99001       100



In [30]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples
fake_labels = np.asarray(fake_labels)

In [31]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[29  1  0]
 [ 2 25  3]
 [ 5  4 21]]
              precision    recall  f1-score   support

           0    0.80556   0.96667   0.87879        30
           1    0.83333   0.83333   0.83333        30
           2    0.87500   0.70000   0.77778        30

    accuracy                        0.83333        90
   macro avg    0.83796   0.83333   0.82997        90
weighted avg    0.83796   0.83333   0.82997        90

